# Simulations

This notebook will simulate the behaviour of a decision algorithm, which will decide when to charge and discharge a battery.

It will be presented at least three algorithms:

1. Baseline
    - No batteries in this scenario
2. Baseline with batteries
    - The same as the baseline, but the CER will have batteries
3. Baseline with batteries and AI/ML
    - The final algorithm will use AI/ML to make a decision on the next action.

Furthermore, there must be scenarios to run these algorithms, with low/high production and low/high battery capacity.

Aditional scenarios and algorithms may or may not be added in the future.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import xgboost as xgb

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
TARIFA_SIMPLES = 0.1788
TARIFA_PONTA = 0.2047
TARIFA_FORA_PONTA = 0.1086

## Pre-Processing

In this section, the data will be pre-processed, using the pre-trained ML model, merging prices and weather data.

In [3]:
energy_df = pd.read_csv("../data/BANES_v4_featureimportance.csv")
energy_df.describe()

,energy,energy_lag_1,energy_lag_46,energy_lag_328,energy_lag_334,energy_lag_335,energy_lag_336
count,2.466431e+06,2.466431e+06,2.466431e+06,2.466431e+06,2.466431e+06,2.466431e+06,2.466431e+06
mean,3.342277e+00,3.342276e+00,3.341474e+00,3.338543e+00,3.338542e+00,3.338538e+00,3.338532e+00
std,6.975297e+00,6.975293e+00,6.974289e+00,6.971822e+00,6.971849e+00,6.971854e+00,6.971861e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e-01,2.000000e-01,2.000000e-01,2.000000e-01,2.000000e-01,2.000000e-01,2.000000e-01
50%,7.100000e-01,7.100000e-01,7.100000e-01,7.090000e-01,7.090000e-01,7.090000e-01,7.090000e-01
75%,2.680000e+00,2.680000e+00,2.675000e+00,2.679000e+00,2.679000e+00,2.679000e+00,2.679000e+00
max,6.100000e+01,6.100000e+01,6.100000e+01,6.080000e+01,6.080000e+01,6.080000e+01,6.080000e+01


In [4]:
# pick 10 locations at random
locs = energy_df.sample(10, random_state=1).index.tolist()
locations = energy_df.iloc[locs]["location"]
assert len(locations) == 10 and len(np.unique(locations)) == 10
energy_df = energy_df[energy_df["location"].isin(locations)]
energy_df.head()

,location,time,energy,energy_lag_1,energy_lag_46,energy_lag_328,energy_lag_334,energy_lag_335,energy_lag_336
45650,##Northgate House Electricity Supply 2,2017-01-08 00:00:00+00:00,0.342,0.339,0.332,0.342,0.334,0.335,0.334
45651,##Northgate House Electricity Supply 2,2017-01-08 00:30:00+00:00,0.343,0.342,0.335,0.337,0.333,0.334,0.335
45652,##Northgate House Electricity Supply 2,2017-01-08 01:00:00+00:00,0.342,0.343,0.338,0.336,0.338,0.333,0.334
45653,##Northgate House Electricity Supply 2,2017-01-08 01:30:00+00:00,0.335,0.342,0.341,0.335,0.341,0.338,0.333
45654,##Northgate House Electricity Supply 2,2017-01-08 02:00:00+00:00,0.338,0.335,0.342,0.335,0.345,0.341,0.338


### Merge with AI/ML model

In [5]:
model: xgb.XGBRegressor = pickle.load(open("../models/model_v1.pkl", "rb"))

In [6]:
X = energy_df.drop(["location", "time", "energy"], axis=1)
y = model.predict(X)


In [7]:
energy_df["energy_pred"] = y
energy_pred_df = energy_df[["time", "energy", "energy_pred"]]
energy_pred_df.head()

,time,energy,energy_pred
45650,2017-01-08 00:00:00+00:00,0.342,0.346541
45651,2017-01-08 00:30:00+00:00,0.343,0.350093
45652,2017-01-08 01:00:00+00:00,0.342,0.347754
45653,2017-01-08 01:30:00+00:00,0.335,0.350093
45654,2017-01-08 02:00:00+00:00,0.338,0.346541


#### Group the community

In [8]:
energy_pred_df = energy_pred_df.groupby("time").sum().reset_index()
energy_pred_df.describe()

,energy,energy_pred
count,54049.000000,54049.000000
mean,20.287596,20.307133
std,10.314216,10.019979
min,2.506500,2.833241
25%,12.825000,12.962095
50%,17.028000,17.352509
75%,25.407000,25.373648
max,76.893000,74.821167


### Merge with weather data

In [9]:
solar_df = pd.read_csv("../data/Solcast_PT30M.csv")
solar_df["PeriodStart"] = pd.to_datetime(solar_df["PeriodStart"], format="%Y-%m-%d %H:%M:%S", utc=True)
solar_df["PeriodEnd"] = pd.to_datetime(solar_df["PeriodEnd"], format="%Y-%m-%d %H:%M:%S", utc=True)

energy_pred_df["time"] = pd.to_datetime(energy_pred_df["time"], format="%Y-%m-%d %H:%M:%S", utc=True)
solar_df.rename(columns={"PeriodEnd": "time"}, inplace=True)
energy_pred_df = energy_pred_df.merge(solar_df, on="time", how="left")
energy_pred_df = energy_pred_df[["time", "energy", "energy_pred", "Dni"]]
energy_pred_df.head()

,time,energy,energy_pred,Dni
0,2017-01-08 00:00:00+00:00,12.1005,12.285041,0
1,2017-01-08 00:30:00+00:00,15.5335,13.659566,0
2,2017-01-08 01:00:00+00:00,15.8120,15.108760,0
3,2017-01-08 01:30:00+00:00,15.2760,15.378116,0
4,2017-01-08 02:00:00+00:00,15.6635,14.970471,0


In [10]:
energy_pred_df["Dni"] /= 1000
energy_pred_df.describe()

,energy,energy_pred,Dni
count,54049.000000,54049.000000,54049.000000
mean,20.287596,20.307133,0.098700
std,10.314216,10.019979,0.221135
min,2.506500,2.833241,0.000000
25%,12.825000,12.962095,0.000000
50%,17.028000,17.352509,0.000000
75%,25.407000,25.373648,0.017000
max,76.893000,74.821167,0.945000


### Energy Production

Use the following formula to calculate the energy production:

$E(t)=A*r(t)*G(t)*\rho$

Where:
- $E(t)$ is the energy production at time $t$;
- $A$ is the area of the solar panel;
- $r(t)$ is the yield of the solar panel at time $t$;
- $G(t)$ is the solar irradiance at time $t$;
- $\rho$ is the performance of the solar panel.

In [11]:
LOW_PRODUCTION = 18 * 1.6 * 0.16 * 0.95
HIGH_PRODUCTION = 10 * 18 * 1.6 * 0.16 * 0.95
AVG_PRODUCTION = (LOW_PRODUCTION + HIGH_PRODUCTION) / 2

In [12]:
baseline = energy_pred_df.copy()
baseline["high_production"] = HIGH_PRODUCTION * baseline["Dni"]
baseline["low_production"] = LOW_PRODUCTION * baseline["Dni"]
baseline["avg_production"] = AVG_PRODUCTION * baseline["Dni"]
baseline = baseline.drop(["Dni"], axis=1)
baseline.describe()

,energy,energy_pred,high_production,low_production,avg_production
count,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000
mean,20.287596,20.307133,4.320677,0.432068,2.376372
std,10.314216,10.019979,9.680403,0.968040,5.324222
min,2.506500,2.833241,0.000000,0.000000,0.000000
25%,12.825000,12.962095,0.000000,0.000000,0.000000
50%,17.028000,17.352509,0.000000,0.000000,0.000000
75%,25.407000,25.373648,0.744192,0.074419,0.409306
max,76.893000,74.821167,41.368320,4.136832,22.752576


## Algoritmo Baseline

Tarifas

In [13]:
from datetime import timedelta, datetime
import pytz

def tarifa_bihoraria_diaria_ponta(time) -> bool:
    return time.hour >= 8 and time.hour < 22

def tarifa_bihoraria_semanal_ponta(time: datetime) -> bool:
    time = time.astimezone(pytz.timezone("Europe/Lisbon"))

    hour = time.hour
    # if it is daylight savings time
    if time.tz.dst != timedelta(0):
        if time.weekday() == 5:
            # it is saturday
            if hour < 9 or (hour >= 14 and hour < 20) or (hour >= 22):
                tarifa = False
            else:
                tarifa = True
        elif time.weekday() == 6:
            # it is sunday
            tarifa = False
        else:
            if hour < 7:
                tarifa = False
            else:
                tarifa = True
    else:
        if time.weekday() == 5:
            # it is saturday
            if (
                time.to_pydatetime()
                < datetime(
                    time.year,
                    time.month,
                    time.day,
                    9,
                    30,
                    0,
                    tzinfo=pytz.timezone("Europe/Lisbon"),
                )
                or (
                    time.to_pydatetime()
                    >= datetime(
                        time.year,
                        time.month,
                        time.day,
                        13,
                        0,
                        0,
                        tzinfo=pytz.timezone("Europe/Lisbon"),
                    )
                    and time.to_pydatetime()
                    < datetime(
                        time.year,
                        time.month,
                        time.day,
                        18,
                        30,
                        0,
                        tzinfo=pytz.timezone("Europe/Lisbon"),
                    )
                )
                or (
                    time.to_pydatetime()
                    >= datetime(
                        time.year,
                        time.month,
                        time.day,
                        22,
                        0,
                        0,
                        tzinfo=pytz.timezone("Europe/Lisbon"),
                    )
                )
            ):
                tarifa = False
            else:
                tarifa = True
        elif time.weekday() == 6:
            # it is sunday
            tarifa = False
        else:
            if hour < 7:
                tarifa = False
            else:
                tarifa = True
    return tarifa


Algoritmo Baseline:
- No batteries in this scenario
- No AI/ML
- Consume production
- Fetch grid if still needed

In [14]:
def baseline_algorithm(line: pd.array) -> pd.array:
    """
    Baseline algorithm. Does not use batteries
    """
    to_consume = line["energy"]

    if to_consume > line["production"]:
        # there is NOT excess
        # consume all the energy produced
        to_consume -= line["production"]
        line["consumed_from_production"] = line["production"]
    else:
        # there is excess
        # consume the necessary energy produced
        line["consumed_from_production"] = to_consume
        to_consume = 0
    # fetch from grid the rest of the energy needed
    line["consumed_from_grid"] = to_consume

    # add prices
    tarifa_diario = TARIFA_PONTA if tarifa_bihoraria_diaria_ponta(line["time"]) else TARIFA_FORA_PONTA
    line["price_bihorario_diario"] = tarifa_diario * line["consumed_from_grid"]
    tarifa_semanal = TARIFA_PONTA if tarifa_bihoraria_semanal_ponta(line["time"]) else TARIFA_FORA_PONTA
    line["price_bihorario_semanal"] = tarifa_semanal * line["consumed_from_grid"]
    line["price_simples"] = TARIFA_SIMPLES * line["consumed_from_grid"]
    return line


In [15]:
low_prod_baseline = baseline.rename(columns={"low_production": "production"})
high_prod_baseline = baseline.rename(columns={"high_production": "production"})
avg_prod_baseline = baseline.rename(columns={"avg_production": "production"})

low_prod_baseline.drop(["high_production", "avg_production"], axis=1, inplace=True)
high_prod_baseline.drop(["low_production", "avg_production"], axis=1, inplace=True)
avg_prod_baseline.drop(["low_production", "high_production"], axis=1, inplace=True)

In [16]:
low_prod_baseline = low_prod_baseline.apply(baseline_algorithm, axis=1)
high_prod_baseline = high_prod_baseline.apply(baseline_algorithm, axis=1)
avg_prod_baseline = avg_prod_baseline.apply(baseline_algorithm, axis=1)


In [17]:
print("Baseline algorithm:")
print("High production:")
print(high_prod_baseline[["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))

print("Low production:")
print(low_prod_baseline[ ["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))

print("Avg production:")
print(avg_prod_baseline[ ["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))

Baseline algorithm:
High production:
price_bihorario_diario     155352.4996
price_bihorario_semanal    155430.4499
price_simples              163012.7337
consumed_from_grid         911704.3270
dtype: object
Low production:
price_bihorario_diario      187225.5356
price_bihorario_semanal     186133.8949
price_simples               191883.2637
consumed_from_grid         1073172.6156
dtype: object
Avg production:
price_bihorario_diario     167865.9807
price_bihorario_semanal    167821.7291
price_simples              174327.7718
consumed_from_grid         974987.5379
dtype: object


## Baseline + Battery Algorithm

Puts the excess energy in the battery and fetches from it if needed.

In [18]:
efficiency = 0.982
BATTERY_CAPACITY = 64

In [19]:
def baseline_battery_algorithm(line: pd.array, state_of_charge: float) -> tuple[pd.array, float]:
    """
    Uses the battery when there is excess or lack of energy. 
    Only if there is still energy missing, consume from grid.
    """
    line["battery_charge"] = state_of_charge
    # energy predicted to consume
    to_consume = line["energy"]

    # there is NOT excess
    if line["production"] < line["energy"]:
        # consume the energy produced
        to_consume -= line["production"]
        line["consumed_from_production"] = line["production"]

        # go fetch the rest to the battery
        if to_consume > line["battery_charge"] * efficiency:
            # if the battery energy is not enough 
            # consume all the energy from the battery
            to_consume -= line["battery_charge"] * efficiency
            line["consumed_from_battery"] = line["battery_charge"] * efficiency
            # add loss from the battery
            line["losses"] = line["battery_charge"] * (1 - efficiency)
            line["battery_charge"] = 0
        else:
            # if the battery energy is enough
            line["battery_charge"] -= to_consume / efficiency
            line["consumed_from_battery"] = to_consume
            # add loss from the battery
            line["losses"] = to_consume * (1 - efficiency) / efficiency
            to_consume = 0
    else:
        # there is excess
        # consume the energy predicted to consume
        line["consumed_from_production"] = to_consume
        to_consume = 0

        # empty space in the battery
        possible_charge = BATTERY_CAPACITY - line["battery_charge"]
        # add the produced energy to the battery
        line["battery_charge"] += min((line["production"] - line["energy"]) * efficiency, possible_charge)
        # log the energy saved in the battery
        line["saved_to_battery"] += min((line["production"] - line["energy"]) * efficiency, possible_charge)
        line["losses"] += line["saved_to_battery"] * (1 - efficiency) / efficiency
        # send the rest of the energy to the grid
        line["send_to_grid"] = line["production"] - line["saved_to_battery"] - line["losses"]
    
    # fetch the rest of the energy to the grid
    line["consumed_from_grid"] += to_consume
    state_of_charge = line["battery_charge"]

    # add prices
    tarifa_diario = TARIFA_PONTA if tarifa_bihoraria_diaria_ponta(line["time"]) else TARIFA_FORA_PONTA
    line["price_bihorario_diario"] = tarifa_diario * line["consumed_from_grid"]
    tarifa_semanal = TARIFA_PONTA if tarifa_bihoraria_semanal_ponta(line["time"]) else TARIFA_FORA_PONTA
    line["price_bihorario_semanal"] = tarifa_semanal * line["consumed_from_grid"]
    line["price_simples"] = TARIFA_SIMPLES * line["consumed_from_grid"]
    return line, state_of_charge


In [20]:
low_prod_battery = low_prod_baseline.copy()
high_prod_battery = high_prod_baseline.copy()
avg_prod_battery = avg_prod_baseline.copy()

low_prod_battery["consumed_from_grid"] = 0
low_prod_battery["consumed_from_production"] = 0
low_prod_battery["consumed_from_battery"] = 0
low_prod_battery["saved_to_battery"] = 0
low_prod_battery["losses"] = 0
low_prod_battery["send_to_grid"] = 0
low_prod_battery["price_bihorario_diario"] = 0
low_prod_battery["price_bihorario_semanal"] = 0
low_prod_battery["price_simples"] = 0
low_prod_battery["battery_charge"] = 0

high_prod_battery["consumed_from_grid"] = 0
high_prod_battery["consumed_from_production"] = 0
high_prod_battery["consumed_from_battery"] = 0
high_prod_battery["saved_to_battery"] = 0
high_prod_battery["losses"] = 0
high_prod_battery["send_to_grid"] = 0
high_prod_battery["price_bihorario_diario"] = 0
high_prod_battery["price_bihorario_semanal"] = 0
high_prod_battery["price_simples"] = 0
high_prod_battery["battery_charge"] = 0

avg_prod_battery["consumed_from_grid"] = 0
avg_prod_battery["consumed_from_production"] = 0
avg_prod_battery["consumed_from_battery"] = 0
avg_prod_battery["saved_to_battery"] = 0
avg_prod_battery["losses"] = 0
avg_prod_battery["send_to_grid"] = 0
avg_prod_battery["price_bihorario_diario"] = 0
avg_prod_battery["price_bihorario_semanal"] = 0
avg_prod_battery["price_simples"] = 0
avg_prod_battery["battery_charge"] = 0

state_of_charge1 = 0
state_of_charge2 = 0
state_of_charge3 = 0

for i in tqdm(range(len(low_prod_battery))):
    low_prod_battery.iloc[i], state_of_charge1 = baseline_battery_algorithm(
        low_prod_battery.iloc[i], state_of_charge1
    )
    high_prod_battery.iloc[i], state_of_charge2 = baseline_battery_algorithm(
        high_prod_battery.iloc[i], state_of_charge2
    )
    avg_prod_battery.iloc[i], state_of_charge3 = baseline_battery_algorithm(
        avg_prod_battery.iloc[i], state_of_charge3
    )


100%|██████████| 54049/54049 [25:12<00:00, 35.73it/s]


In [21]:
high_prod_battery.describe()

,energy,energy_pred,production,consumed_from_production,consumed_from_grid,price_bihorario_diario,price_bihorario_semanal,price_simples,consumed_from_battery,saved_to_battery,losses,send_to_grid,battery_charge
count,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000,54049.000000
mean,20.287596,20.307133,4.320677,3.419489,16.711502,2.842398,2.848493,2.988017,0.156605,0.159476,0.005794,2.433669,1.321909
std,10.314216,10.019980,9.680403,7.618918,11.093781,2.411879,2.428476,1.983568,1.086104,1.049612,0.027381,8.164575,4.182031
min,2.506500,2.833241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12.825000,12.962095,0.000000,0.000000,11.026948,1.302331,1.280177,1.971618,0.000000,0.000000,0.000000,0.000000,0.000000
50%,17.028000,17.352509,0.000000,0.000000,14.833000,2.062857,2.020177,2.652140,0.000000,0.000000,0.000000,0.000000,0.000000
75%,25.407000,25.373648,0.744192,0.744192,21.691500,3.934642,4.047448,3.878440,0.000000,0.000000,0.000000,0.000000,0.000000
max,76.893000,74.821167,41.368320,39.704832,76.893000,15.739997,15.739997,13.748468,15.712000,16.000000,0.293279,41.368320,16.000000


In [22]:
print("Baseline + Battery")
print("High Production:")
print(high_prod_battery[["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))
print("Low Production:")
print(low_prod_battery[["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))
print("Average Production:")
print(avg_prod_battery[["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))


Baseline + Battery
High Production:
price_bihorario_diario     153628.7923
price_bihorario_semanal    153958.2161
price_simples              161499.3063
consumed_from_grid         903239.9682
dtype: object
Low Production:
price_bihorario_diario      187225.3102
price_bihorario_semanal     186133.7753
price_simples               191883.0668
consumed_from_grid         1073171.5146
dtype: object
Average Production:
price_bihorario_diario     167369.0751
price_bihorario_semanal    167464.5900
price_simples              173891.9425
consumed_from_grid         972550.0141
dtype: object


## Baseline + Battery + AI/ML Algorithm

### Midnight Consumption Prediction

Predict the consumption at midnight and then use it to predict the next half hour consumption.

In [23]:
midnight_consumption = energy_df.copy()

midnight_consumption.drop(["energy_pred"], axis=1, inplace=True)

midnight_consumption["time"] = pd.to_datetime(midnight_consumption["time"])

# # select rows at midnight
X = midnight_consumption.loc[(midnight_consumption["time"].dt.hour == 0) & (midnight_consumption["time"].dt.minute == 0)]
X = X.drop(["time", "energy", "location"], axis=1)

midnight_consumption.loc[(midnight_consumption["time"].dt.hour == 0) & (midnight_consumption["time"].dt.minute == 0), "energy_pred"] = model.predict(X)
midnight_consumption.head()


,location,time,energy,energy_lag_1,energy_lag_46,energy_lag_328,energy_lag_334,energy_lag_335,energy_lag_336,energy_pred
45650,##Northgate House Electricity Supply 2,2017-01-08 00:00:00+00:00,0.342,0.339,0.332,0.342,0.334,0.335,0.334,0.346541
45651,##Northgate House Electricity Supply 2,2017-01-08 00:30:00+00:00,0.343,0.342,0.335,0.337,0.333,0.334,0.335,NaN
45652,##Northgate House Electricity Supply 2,2017-01-08 01:00:00+00:00,0.342,0.343,0.338,0.336,0.338,0.333,0.334,NaN
45653,##Northgate House Electricity Supply 2,2017-01-08 01:30:00+00:00,0.335,0.342,0.341,0.335,0.341,0.338,0.333,NaN
45654,##Northgate House Electricity Supply 2,2017-01-08 02:00:00+00:00,0.338,0.335,0.342,0.335,0.345,0.341,0.338,NaN


In [24]:
# shift energy 1
midnight_consumption["energy_shift_1"] = midnight_consumption["energy_pred"].shift(1)

# predict the full day
for i in tqdm(range(1, 48)):
    X = midnight_consumption.loc[
        (midnight_consumption["time"].dt.hour == i // 2)  # hour
        & (midnight_consumption["time"].dt.minute == i % 2 * 30)  # minute
    ]
    if i >= 46:  # use energy shift 46, instead of real energy, because the that part of the day is not available yet
        midnight_consumption["energy_shift_46"] = midnight_consumption["energy_pred"].shift(46)
        X = midnight_consumption.loc[
            (midnight_consumption["time"].dt.hour == i // 2)  # hour
            & (midnight_consumption["time"].dt.minute == i % 2 * 30)  # minute
        ]

        X = X[
            [
                "energy_shift_1",
                "energy_shift_46",
                "energy_lag_328",
                "energy_lag_328",
                "energy_lag_335",
                "energy_lag_336",
            ]
        ]
    else:
        X = X[
            [
                "energy_shift_1",
                "energy_lag_46",
                "energy_lag_328",
                "energy_lag_328",
                "energy_lag_335",
                "energy_lag_336",
            ]
        ]
    midnight_consumption.loc[
        (midnight_consumption["time"].dt.hour == i // 2)
        & (midnight_consumption["time"].dt.minute == i % 2 * 30),
        "energy_pred",
    ] = model.predict(X)
    midnight_consumption["energy_shift_1"] = midnight_consumption["energy_pred"].shift(1)

midnight_consumption.head()


100%|██████████| 47/47 [00:13<00:00,  3.40it/s]


,location,time,energy,energy_lag_1,energy_lag_46,energy_lag_328,energy_lag_334,energy_lag_335,energy_lag_336,energy_pred,energy_shift_1,energy_shift_46
45650,##Northgate House Electricity Supply 2,2017-01-08 00:00:00+00:00,0.342,0.339,0.332,0.342,0.334,0.335,0.334,0.346541,NaN,NaN
45651,##Northgate House Electricity Supply 2,2017-01-08 00:30:00+00:00,0.343,0.342,0.335,0.337,0.333,0.334,0.335,0.350093,0.346541,NaN
45652,##Northgate House Electricity Supply 2,2017-01-08 01:00:00+00:00,0.342,0.343,0.338,0.336,0.338,0.333,0.334,0.347754,0.350093,NaN
45653,##Northgate House Electricity Supply 2,2017-01-08 01:30:00+00:00,0.335,0.342,0.341,0.335,0.341,0.338,0.333,0.350093,0.347754,NaN
45654,##Northgate House Electricity Supply 2,2017-01-08 02:00:00+00:00,0.338,0.335,0.342,0.335,0.345,0.341,0.338,0.350093,0.350093,NaN


In [25]:
midnight_consumption["time"] = pd.to_datetime(midnight_consumption["time"])
cons_m1 = midnight_consumption[(midnight_consumption["time"].dt.hour >= 9) & (midnight_consumption["time"].dt.hour <= 18)]
cons_m1 = cons_m1.groupby(cons_m1["time"].dt.date).sum()
cons_m1 = cons_m1[["energy_pred"]]

cons_m1.head()

,energy_pred
time,
2017-01-08,260.738770
2017-01-09,286.047119
2017-01-10,655.025269
2017-01-11,728.638794
2017-01-12,726.581177


In [26]:
cons_m2 = midnight_consumption[(midnight_consumption.time.dt.hour >= 6) & (midnight_consumption.time.dt.hour <= 9)]
cons_m2 = cons_m2.groupby(cons_m2.time.dt.date).sum()
cons_m2 = cons_m2[["energy_pred"]]

cons_m2.head()


,energy_pred
time,
2017-01-08,118.441208
2017-01-09,130.730865
2017-01-10,204.125427
2017-01-11,210.173065
2017-01-12,218.266968


In [27]:
# sum the energy in a day
consumption_pred = midnight_consumption.groupby(midnight_consumption["time"].dt.date).sum()
midnight_consumption.drop("time", axis=1, inplace=True)
consumption_pred.head()


,energy,energy_lag_1,energy_lag_46,energy_lag_328,energy_lag_334,energy_lag_335,energy_lag_336,energy_pred,energy_shift_1,energy_shift_46
time,,,,,,,,,,
2017-01-08,704.6990,704.8260,776.9045,636.7765,626.2460,623.0270,621.4795,666.873413,669.836426,1377.887817
2017-01-09,1212.0825,1213.1880,704.6655,737.2205,736.7910,737.9110,737.8375,719.015198,718.169800,656.380249
2017-01-10,1271.3735,1270.3910,1210.5335,1141.2470,1140.7555,1141.0375,1140.6610,1149.309204,1149.679810,704.913452
2017-01-11,1152.8515,1152.8605,1271.6145,1215.4875,1212.2645,1211.7735,1210.8200,1229.814575,1228.956787,1137.582275
2017-01-12,1249.3505,1247.8710,1153.0790,1287.2605,1285.9060,1285.1790,1285.8375,1249.597656,1249.213135,1216.762085


In [28]:
consumption_pred = consumption_pred[["energy", "energy_pred"]]
consumption_pred.head()

,energy,energy_pred
time,,
2017-01-08,704.6990,666.873413
2017-01-09,1212.0825,719.015198
2017-01-10,1271.3735,1149.309204
2017-01-11,1152.8515,1229.814575
2017-01-12,1249.3505,1249.597656


In [29]:
production_pred = energy_pred_df.groupby(energy_pred_df["time"].dt.date).sum()
production_pred.head()

,energy,energy_pred,Dni
time,,,
2017-01-08,704.6990,693.797485,0.311
2017-01-09,1212.0825,1076.781860,0.126
2017-01-10,1271.3735,1238.020996,0.000
2017-01-11,1152.8515,1178.930542,3.595
2017-01-12,1249.3505,1252.612061,0.000


In [30]:
low_production_pred = production_pred.copy()
high_production_pred = production_pred.copy()
avg_production_pred = production_pred.copy()

low_production_pred["production"] = low_production_pred["Dni"] * LOW_PRODUCTION
high_production_pred["production"] = high_production_pred["Dni"] * HIGH_PRODUCTION
avg_production_pred["production"] = avg_production_pred["Dni"] * AVG_PRODUCTION

low_production_pred = low_production_pred[["production"]]
high_production_pred = high_production_pred[["production"]]
avg_production_pred = avg_production_pred[["production"]]

In [31]:
high_production_pred.describe()

,production
count,1127.000000
mean,207.212317
std,239.687603
min,0.000000
25%,13.351680
50%,111.103488
75%,330.859008
max,1015.034112


In [32]:
# parse the index to string
low_production_pred.index = low_production_pred.index.astype(str)
high_production_pred.index = high_production_pred.index.astype(str)
avg_production_pred.index = avg_production_pred.index.astype(str)

consumption_pred.index = consumption_pred.index.astype(str)

cons_m1.index = cons_m1.index.astype(str)
cons_m2.index = cons_m2.index.astype(str)

high_production_pred.head()

,production
time,
2017-01-08,13.614336
2017-01-09,5.515776
2017-01-10,0.000000
2017-01-11,157.374720
2017-01-12,0.000000


### Algorithm

In [33]:
def battery_ml_algorithm(line: pd.array, state_of_charge: float, production_pred: pd.DataFrame, tarifa) -> tuple[pd.array, float]:
    """
    Battery ML algorithm
    """
    # battery available
    battery_available = BATTERY_CAPACITY - state_of_charge
    day = str(line["time"].strftime("%Y-%m-%d"))
    if line.time.hour == 0 and line.time.minute == 0:
        # it is midnight
        consumption_prediction = consumption_pred.loc[day]["energy_pred"]
        production_prediction = production_pred.loc[day]["production"]

        m1 = cons_m1.loc[day]["energy_pred"] / consumption_prediction
        m2 = cons_m2.loc[day]["energy_pred"] / consumption_prediction

        if production_prediction > consumption_prediction * m1:
            # production is greater than consumption

            excess = production_prediction - consumption_prediction * m1

            if excess < battery_available:
                # battery is NOT full
                # charge battery with the excess
                state_of_charge += battery_available - excess
                line["consumed_from_grid"] += (battery_available - excess) / efficiency
                line["saved_to_battery"] += battery_available - excess
                line["losses"] += (
                    (battery_available - excess) * (1 - efficiency) / efficiency
                )
            else:
                # battery will be full
                # charge battery with the consumption in the morning
                state_of_charge += min(consumption_prediction * m2, battery_available)
                line["consumed_from_grid"] += min(
                    consumption_prediction * m2, battery_available
                ) / efficiency
                line["saved_to_battery"] += min(
                    consumption_prediction * m2, battery_available
                )
                line["losses"] += min(consumption_prediction * m2, battery_available) * (
                    1 - efficiency
                ) / efficiency
        else:
            # production is smaller than consumption
            # pre-charge battery to its full capacity
            state_of_charge += battery_available
            line["consumed_from_grid"] += battery_available / efficiency
            line["saved_to_battery"] += battery_available
            line["losses"] += battery_available * (1 - efficiency) / efficiency

    if not tarifa(line.time):
        # not hora de ponta

        to_consume = line["energy"]
        # only consume from production and grid
        # if the energy predicted to consume is greater than the energy produced
        if line["production"] < line["energy"]:
            # consume the energy produced
            to_consume -= line["production"]
            line["consumed_from_production"] += line["production"]
        else:
            # if the energy predicted to consume is less than the energy produced
            # consume the energy predicted to consume
            line["consumed_from_production"] = to_consume
            to_consume = 0

            # add the produced energy to the battery
            state_of_charge += min(
                (line["production"] - line["energy"]) * efficiency, battery_available
            )
            # log the energy saved in the battery
            line["saved_to_battery"] += min(
                (line["production"] - line["energy"]) * efficiency, battery_available
            )
            # log the losses
            line["losses"] += line["saved_to_battery"] * (1 - efficiency) / efficiency
            # send the rest of the energy to the grid
            line["send_to_grid"] = line["production"] - line["saved_to_battery"] - line["losses"]

        # fetch the rest of the energy to the grid
        line["consumed_from_grid"] += to_consume
        line["battery_charge"] = state_of_charge

        # add prices
        tarifa_diario = TARIFA_PONTA if tarifa_bihoraria_diaria_ponta(line["time"]) else TARIFA_FORA_PONTA
        line["price_bihorario_diario"] = tarifa_diario * line["consumed_from_grid"]
        tarifa_semanal = TARIFA_PONTA if tarifa_bihoraria_semanal_ponta(line["time"]) else TARIFA_FORA_PONTA
        line["price_bihorario_semanal"] = tarifa_semanal * line["consumed_from_grid"]
        line["price_simples"] = TARIFA_SIMPLES * line["consumed_from_grid"]
        return line, state_of_charge

    return baseline_battery_algorithm(line, state_of_charge)



In [34]:
low_prod_battery_ml = low_prod_battery.copy()
high_prod_battery_ml = high_prod_battery.copy()
avg_prod_battery_ml = avg_prod_battery.copy()

# remove the last row
low_prod_battery_ml = low_prod_battery_ml.iloc[:-1]
high_prod_battery_ml = high_prod_battery_ml.iloc[:-1]
avg_prod_battery_ml = avg_prod_battery_ml.iloc[:-1]

low_prod_battery_ml["consumed_from_grid"] = 0
low_prod_battery_ml["consumed_from_battery"] = 0
low_prod_battery_ml["consumed_from_production"] = 0
low_prod_battery_ml["saved_to_battery"] = 0
low_prod_battery_ml["losses"] = 0
low_prod_battery_ml["send_to_grid"] = 0
low_prod_battery_ml["battery_charge"] = 0

high_prod_battery_ml["consumed_from_grid"] = 0
high_prod_battery_ml["consumed_from_production"] = 0
high_prod_battery_ml["consumed_from_battery"] = 0
high_prod_battery_ml["saved_to_battery"] = 0
high_prod_battery_ml["losses"] = 0
high_prod_battery_ml["send_to_grid"] = 0
high_prod_battery_ml["battery_charge"] = 0

avg_prod_battery_ml["consumed_from_grid"] = 0
avg_prod_battery_ml["consumed_from_production"] = 0
avg_prod_battery_ml["consumed_from_battery"] = 0
avg_prod_battery_ml["saved_to_battery"] = 0
avg_prod_battery_ml["losses"] = 0
avg_prod_battery_ml["send_to_grid"] = 0
avg_prod_battery_ml["battery_charge"] = 0

state_of_charge1 = 0
state_of_charge2 = 0
state_of_charge3 = 0

high_prod_battery_ml["time"] = pd.to_datetime(high_prod_battery_ml["time"])
low_prod_battery_ml["time"] = pd.to_datetime(low_prod_battery_ml["time"])
avg_prod_battery_ml["time"] = pd.to_datetime(avg_prod_battery_ml["time"])

for i in tqdm(range(len(low_prod_battery_ml))):
    high_prod_battery_ml.iloc[i], state_of_charge1 = battery_ml_algorithm(
        high_prod_battery_ml.iloc[i],
        state_of_charge1,
        high_production_pred,
        lambda time: time.hour >= 10,
    )
    low_prod_battery_ml.iloc[i], state_of_charge2 = battery_ml_algorithm(
        low_prod_battery_ml.iloc[i],
        state_of_charge2,
        low_production_pred,
        lambda time: time.hour >= 10,
    )
    avg_prod_battery_ml.iloc[i], state_of_charge3 = battery_ml_algorithm(
        avg_prod_battery_ml.iloc[i],
        state_of_charge3,
        avg_production_pred,
        lambda time: time.hour >= 10,
    )


100%|██████████| 54048/54048 [14:27<00:00, 62.32it/s]  


In [35]:
print("Baseline + Battery + ML")
print("High Production:")
print(high_prod_battery_ml[["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))
print("Low Production:")
print(low_prod_battery_ml[["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))
print("Average Production:")
print(avg_prod_battery_ml[["price_bihorario_diario", "price_bihorario_semanal", "price_simples", "consumed_from_grid"]].sum().apply(lambda x: f"{x:.4f}"))


Baseline + Battery + ML
High Production:
price_bihorario_diario     152887.6391
price_bihorario_semanal    153374.2122
price_simples              162393.1771
consumed_from_grid         908239.2454
dtype: object
Low Production:
price_bihorario_diario      185595.7721
price_bihorario_semanal     184747.3349
price_simples               191999.3617
consumed_from_grid         1073821.9333
dtype: object
Average Production:
price_bihorario_diario     165986.0544
price_bihorario_semanal    166269.8564
price_simples              174224.4329
consumed_from_grid         974409.5799
dtype: object


# Sell Grid gains

In [36]:
print("High:", (high_prod_battery["send_to_grid"] * 0.03).sum())
print("Low", (low_prod_battery["send_to_grid"] * 0.03).sum())
print("Avg", (avg_prod_battery["send_to_grid"] * 0.03).sum())

High: 3946.1220377383415
Low 0.8308049999999999
Avg 727.1969097061378


In [37]:
print("High:", (high_prod_battery_ml["send_to_grid"] * 0.03).sum())
print("Low", (low_prod_battery_ml["send_to_grid"] * 0.03).sum())
print("Avg", (avg_prod_battery_ml["send_to_grid"] * 0.03).sum())

High: 4081.492868926227
Low 0.8308049999999999
Avg 764.8219622518583


# No Production Use Case

In [68]:
def no_production(line):
    # add prices
    tarifa_diario = TARIFA_PONTA if tarifa_bihoraria_diaria_ponta(line["time"]) else TARIFA_FORA_PONTA
    line["price_bihorario_diario"] = tarifa_diario * line["energy"]
    tarifa_semanal = TARIFA_PONTA if tarifa_bihoraria_semanal_ponta(line["time"]) else TARIFA_FORA_PONTA
    line["price_bihorario_semanal"] = tarifa_semanal * line["energy"]
    line["price_simples"] = TARIFA_SIMPLES * line["energy"]
    return line

no_prod = high_prod_baseline[["time", "energy"]]
no_prod["time"] = pd.to_datetime(no_prod["time"])
no_prod = no_prod.apply(no_production, axis=1)
no_prod.sum()


energy                     1.096524e+06
price_bihorario_diario     1.918342e+05
price_bihorario_semanal    1.904042e+05
price_simples              1.960585e+05
dtype: float64

In [69]:
no_prod.sum().apply(lambda x: f"{x:.4f}")

energy                     1096524.3020
price_bihorario_diario      191834.1574
price_bihorario_semanal     190404.1557
price_simples               196058.5452
dtype: object